In [1]:
from graphdatascience import GraphDataScience
import emblaze
import numpy as np
import pandas as pd
import json
from web3 import Web3
from solcx import compile_standard, install_solc
import os
from dotenv import load_dotenv
import calendar
import time

which: no solc in (/home/sahilarora/anaconda3/bin:/home/sahilarora/anaconda3/condabin:/home/sahilarora/.cargo/bin:/usr/local/sbin:/usr/local/bin:/usr/bin:/opt/android-sdk/platform-tools:/home/sahilarora/.dotnet/tools:/var/lib/flatpak/exports/bin:/usr/lib/jvm/default/bin:/usr/bin/site_perl:/usr/bin/vendor_perl:/usr/bin/core_perl)


In [2]:
gds = GraphDataScience("bolt://localhost:11003", auth=("neo4j", "1234567890"), database="neo4j")
print(gds.version())
assert gds.version()
using_enterprise = gds.is_licensed()

2.0.3


In [3]:
def setup_graph():
    gds.run_cypher("""
match (:Airport)-[r:HAS_ROUTE]->(:Airport)
with collect(r) as routes, max(r.distance) as maxDistance
foreach(route in routes | set route.weight = maxDistance + 1 - route.distance)
""")
    gds.run_cypher("""
match (n:Airport)
with collect(n) as airports
foreach(airport in airports | set airport.longitude = airport.location.x)
""")
    gds.run_cypher("""
match (n:Airport)
with collect(n) as airports
foreach(airport in airports | set airport.latitude = airport.location.y)
""")
    gds.run_cypher("""
match (p:Airport)-[r:HAS_ROUTE]->(q:Airport)
SET r.avgPageRank = p.pagerank + q.pagerank
""")
    gds.run_cypher("""
match (:Airport)-[r:HAS_ROUTE]->(:Airport)
with collect(r) as routes, max(r.avgPageRank) as maxRank
foreach(route in routes | set route.weightedRank = maxRank + 1 - route.avgPageRank)
""")

In [4]:


G_routes, result = gds.graph.project(
   "air-routes", 
   "Airport",                                   
   {"HAS_ROUTE":
      {"orientation":"UNDIRECTED", 
      "aggregation":"MAX"}
   }, 
   relationshipProperties = ["weight", "distance", "weightedRank"],
    nodeProperties = ["latitude", "longitude"]
)

routes_wcc = gds.wcc.stats(G_routes)
routes_wcc['componentDistribution']

gds.wcc.mutate(G_routes, mutateProperty = 'componentId')

gds.graph.writeNodeProperties(G_routes, ['componentId'])

gds.run_cypher("MATCH (a:Airport) RETURN a.componentId as componentId, count(*) as nodeCount ORDER BY count(*) DESC limit 1")

G_connected_airports, result = gds.beta.graph.project.subgraph("connected-airports", G_routes, "n.componentId = 0", "*")

In [3]:
def dijkstra_distance(source, destination):
    return gds.run_cypher("""
MATCH (p1:Airport {iata:'""" + source +"""'}), (p2:Airport {iata:'""" + destination + """'})

CALL gds.shortestPath.dijkstra.stream('air-routes',{
    sourceNode: p1,
    targetNode: p2,
    relationshipWeightProperty: 'distance'
})
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
RETURN
    index,
    gds.util.asNode(sourceNode).city AS sourceNodeName,
    gds.util.asNode(targetNode).city AS targetNodeName,
    totalCost,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).city] AS nodeNames,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).iata] AS nodeIATA,
    costs,
    nodes(path) as path
ORDER BY index
""")
def astar_distance(source, destination):
    return gds.run_cypher("""
MATCH (p1:Airport {iata:'""" + source + """'}), (p2:Airport {iata:'""" + destination + """'})

CALL gds.shortestPath.astar.stream('air-routes',{
    sourceNode: p1,
    targetNode: p2,
    latitudeProperty: 'latitude',
    longitudeProperty: 'longitude',
    relationshipWeightProperty: 'distance'
})
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
RETURN
    index,
    gds.util.asNode(sourceNode).city AS sourceNodeName,
    gds.util.asNode(targetNode).city AS targetNodeName,
    totalCost,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).city] AS nodeNames,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).iata] AS nodeIATA,
    costs,
    nodes(path) as path
ORDER BY index
""")
def dijkstra_rank(source, destination):
    return gds.run_cypher("""
MATCH (p1:Airport {iata:'""" + source + """'}), (p2:Airport {iata:'""" + destination + """'})

CALL gds.shortestPath.dijkstra.stream('air-routes',{
    sourceNode: p1,
    targetNode: p2,
    relationshipWeightProperty: 'weightedRank'
})
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
RETURN
    index,
    gds.util.asNode(sourceNode).city AS sourceNodeName,
    gds.util.asNode(targetNode).city AS targetNodeName,
    totalCost,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).city] AS nodeNames,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).iata] AS nodeIATA,
    costs,
    nodes(path) as path
ORDER BY index
""")

In [4]:
def setupContract():
    nonce = w3.eth.getTransactionCount(my_address)
    tx = SimpleStorage.constructor().buildTransaction(
    {
        "chainId": chain_id,
        "gasPrice": w3.eth.gas_price,
        "from": my_address,
        "nonce": w3.eth.getTransactionCount(my_address),
    }
    )
    signed_tx = w3.eth.account.signTransaction(tx, private_key=private_key)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    print(f"Contract deployed to {tx_receipt.contractAddress}")
    return tx_receipt.contractAddress

In [5]:
def createSupplyNodeTransaction(itemID, itemName, itemICAO, stepID):
    simple_storage = w3.eth.contract(address=contractAddress, abi=abi)
    new_transaction = simple_storage.functions.create_store_event(itemID, itemName, itemICAO, stepID).buildTransaction(
    {
        "chainId": chain_id,
        "gasPrice": w3.eth.gas_price,
        "from": my_address,
        "nonce": w3.eth.getTransactionCount(my_address),
    }
    )
    signed_new_txn = w3.eth.account.sign_transaction(
    new_transaction, private_key=private_key
    )
    tx_new_hash = w3.eth.send_raw_transaction(signed_new_txn.rawTransaction)
    print("Creating a supply transaction for consignment ID: " + str(itemID) + ", location: " + itemICAO)
    tx_new_receipt = w3.eth.wait_for_transaction_receipt(tx_new_hash)
    print(tx_new_receipt)
    
    

In [6]:
def printAllContractData():
    simple_storage = w3.eth.contract(address=contractAddress, abi=abi)
    list_el = simple_storage.functions.retrieveLog().call()
    row = "{name1:^20}|{name2:^20}|{name3:^20}|{name4:^20}".format
    print(row(name1="Consignment ID", name2="Consignment Name", name3="Location", name4="PathID"))

    for tup in list_el:
        print(row(name1=tup[0], name2=tup[1], name3=tup[2], name4=tup[3]))
def printbyIDContractData(itemID):
    simple_storage = w3.eth.contract(address=contractAddress, abi=abi)
    list_el = simple_storage.functions.retrieveLog().call()
    list_el_filtered = []
    addresses = []
    for i in list_el:
        if i[0] == itemID:
            addresses.append(i[2])
            list_el_filtered.append(i)
    row = "{name1:^20}|{name2:^20}|{name3:^20}|{name4:^20}".format
    print(*addresses, sep=" -----> ")
    
    print(row(name1="Consignment ID", name2="Consignment Name", name3="Location", name4="PathID"))

    for tup in list_el_filtered:
        print(row(name1=tup[0], name2=tup[1], name3=tup[2], name4=tup[3]))

In [7]:
with open(
    "/home/sahilarora/capstone-19BCE1366-2023/contract/SimpleStorageContract.sol",
    "r",
) as file:
    simple_storage_file = file.read()
install_solc("0.6.0")
compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {"SimpleStorageContract.sol": {"content": simple_storage_file}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    solc_version="0.6.0",
)

with open("compiled_code.json", "w") as file:
    json.dump(compiled_sol, file)
    
bytecode = compiled_sol["contracts"]["SimpleStorageContract.sol"]["SimpleStorage"][
    "evm"
]["bytecode"]["object"]
abi = json.loads(
    compiled_sol["contracts"]["SimpleStorageContract.sol"]["SimpleStorage"]["metadata"]
)["output"]["abi"]

w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))

In [8]:
chain_id = 1337
my_address = "0x67cC7D1dcAE9b1C23Fc4ef2027d6c1B0981E9B32"
# private_key = os.getenv("PRIVATE_KEY")
private_key = "0x12d8bd1785a48823297579bfa4773015bc7d4de689330fb003338eaf9a707b9f"

In [9]:
SimpleStorage = w3.eth.contract(abi=abi, bytecode=bytecode)

In [10]:
contractAddress = setupContract()

Contract deployed to 0xeb4e81A5a117344C483c67AEc767CE268FF1216C


In [11]:
def create_supply_route(source, destination, distance_algo, consignment_name):
    ts = calendar.timegm(time.gmtime())
    print("Consignment_id: " + str(ts))
    route = []
    if distance_algo == "d_distance":
        route = dijkstra_distance(source, destination)
    elif distance_algo == "a_distance":
        route = astar_distance(source, destination)
    else:
        route = dijkstra_rank(source, destination)
    IATAroute = route["nodeIATA"][0]
    
    for i in range(len(IATAroute)):
        createSupplyNodeTransaction(ts, consignment_name, IATAroute[i] , i+1)
    

In [19]:
create_supply_route("DEL", "HPN", "d_distance", "p1")

Consignment_id: 1671698219
Creating a supply transaction for consignment ID: 1671698219, location: DEL
AttributeDict({'transactionHash': HexBytes('0x71fd1a9046de21bafe03a04fd5b9eed73a2deaf8e4284a6e2ebbe69f17778a34'), 'transactionIndex': 0, 'blockNumber': 4, 'blockHash': HexBytes('0xc704b7f36d8c866529cdbd325c685e910b6373fed5edbdc5b61b4d484a42b828'), 'from': '0x67cC7D1dcAE9b1C23Fc4ef2027d6c1B0981E9B32', 'to': '0xeb4e81A5a117344C483c67AEc767CE268FF1216C', 'cumulativeGasUsed': 135395, 'gasUsed': 135395, 'contractAddress': None, 'logs': [], 'logsBloom': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [65]:
create_supply_route("DEL", "HPN", "djikstra_rank", "p1")

Consignment_id: 1670994512
Creating a supply transaction for consignment ID: 1670994512, location: DEL
AttributeDict({'transactionHash': HexBytes('0x46d5cd5d4020dc3be2ecec513664caa02bc27bc75fa64a8c5a60f912ad9b97ea'), 'transactionIndex': 0, 'blockNumber': 49, 'blockHash': HexBytes('0x1f8be9da65bca2eec4b0f6cdbd21cca424f8077edcb820c851cdf141ffb1cc95'), 'from': '0x944336C4b424450A20Bc12E529Fc4999395E88E3', 'to': '0xdD178159eAb4DcBf017AB7A4b52FBb14D8c2cA14', 'cumulativeGasUsed': 118295, 'gasUsed': 118295, 'contractAddress': None, 'logs': [], 'logsBloom': HexBytes('0x000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [20]:
printAllContractData()

   Consignment ID   |  Consignment Name  |      Location      |       PathID       
     1671698219     |         p1         |        DEL         |         1          
     1671698219     |         p1         |        KEF         |         2          
     1671698219     |         p1         |        BOS         |         3          
     1671698219     |         p1         |        HPN         |         4          


In [21]:
printbyIDContractData(1671698219)

DEL -----> KEF -----> BOS -----> HPN
   Consignment ID   |  Consignment Name  |      Location      |       PathID       
     1671698219     |         p1         |        DEL         |         1          
     1671698219     |         p1         |        KEF         |         2          
     1671698219     |         p1         |        BOS         |         3          
     1671698219     |         p1         |        HPN         |         4          
